<h1 align="center">Computación de Alto Desempeño</h1>
<h1 align="center">MPI - Introducción</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

***
|[![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)](mailto:calvarezh@udemedellin.edu.co)||[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/HPC/blob/main/HPC14_MPI_Intro.ipynb)
|-:|:-|--:|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

## **Introducción a MPI (Message Passing Interface)**

### **¿Qué es MPI?**

MPI, o "Interfaz de Paso de Mensajes", es un estándar para la programación de aplicaciones paralelas distribuidas. En lugar de usar un único procesador, MPI permite que múltiples procesadores trabajen de manera coordinada en el procesamiento de un mismo problema. A diferencia de los sistemas de memoria compartida, MPI se utiliza principalmente en arquitecturas de memoria distribuida, donde cada procesador tiene su propio conjunto de memoria.

MPI es compatible con varios lenguajes, incluyendo C, C++, y Fortran. A través de MPI, los procesadores pueden enviar y recibir datos entre sí mediante la técnica de "paso de mensajes".

### **Instalación de MPI**

**En Linux**
Para instalar MPI en una distribución basada en Debian (como Ubuntu), puedes usar el siguiente comando:

```bash
sudo apt-get update
sudo apt-get install mpich
```

Esto instalará MPICH, una implementación popular de MPI. También está disponible OpenMPI, otra implementación que puede ser instalada con:

```bash
sudo apt-get install openmpi-bin openmpi-common libopenmpi-dev
```

**En Windows**
Para instalar MPI en Windows, se puede usar Microsoft MPI (MS-MPI). Sigue los pasos a continuación:

1. Descarga [MS-MPI](https://www.microsoft.com/en-us/download/details.aspx?id=57467) desde el sitio oficial de Microsoft. Se tienen dos archivos: `msmpisetup.exe` y `msmpisdk.msi`. Descárguelos.
2. Instala primero MS-MPI Redistributable (`msmpisetup.exe`) y luego `msmpisdk.msi`.
3. Añade el directorio `C:\Program Files\Microsoft MPI\Bin` a la variable de entorno `PATH`.
4. Verifica la instalación ejecutando en la terminal:

```bash
$set msmpi
$mpiexec
```

### **1. Fundamentos del Paso de Mensajes**

MPI se basa en el concepto de procesos que se comunican entre sí mediante el envío y la recepción de mensajes. Cada proceso tiene un identificador único llamado *rank*, y estos procesos pueden residir en diferentes máquinas.

Un programa MPI típico se inicia con la inicialización de MPI y se cierra limpiamente con una llamada para finalizarlo. Un esqueleto básico de un programa en C es el siguiente:

**Ejemplo básico en C:**

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);  // Inicializa el entorno MPI

    int world_size;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);  // Número total de procesos

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);  // Identificador del proceso actual

    printf("Hello from process %d out of %d!\n", world_rank, world_size);

    MPI_Finalize();  // Finaliza el entorno MPI
    return 0;
}
```

Para ejecutar este código en un entorno MPI, se compila de la siguiente manera:

```bash
mpicc hello_mpi.c -o hello_mpi
```

Y luego se ejecuta con varios procesos:

```bash
mpiexec -n 4 ./hello_mpi
```

Este ejemplo inicializa MPI, identifica cuántos procesos están participando y luego imprime un mensaje desde cada proceso.


### **2. Caja de Herramientas Básica de Paso de Mensajes**

**MPI_Send y MPI_Recv**

Las dos funciones más básicas en MPI son `MPI_Send` y `MPI_Recv`, que permiten el envío y la recepción de mensajes entre procesos. A continuación se presenta un ejemplo simple en el que el proceso con *rank* 0 envía un mensaje al proceso con *rank* 1.

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    if (world_rank == 0) {
        int number = 42;
        MPI_Send(&number, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Proceso 0 envió número %d al proceso 1\n", number);
    } else if (world_rank == 1) {
        int number;
        MPI_Recv(&number, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Proceso 1 recibió número %d desde el proceso 0\n", number);
    }

    MPI_Finalize();
    return 0;
}
```

En este ejemplo, el proceso 0 envía un número entero al proceso 1. Para ejecutarlo con dos procesos, se usa:

```bash
mpiexec -n 2 ./send_recv
```


### **3. Comunicación Punto a Punto (Point-to-Point Communication)**

En MPI, la comunicación punto a punto se refiere al intercambio directo de mensajes entre dos procesos. Esto se realiza principalmente con las funciones `MPI_Send` y `MPI_Recv`, como vimos en el ejemplo anterior. Es importante tener en cuenta los parámetros clave de estas funciones:

- **buffer**: El dato que será enviado o recibido.
- **count**: La cantidad de datos a enviar.
- **datatype**: El tipo de dato, como `MPI_INT`, `MPI_FLOAT`, etc.
- **dest/source**: El proceso destino (en `MPI_Send`) o fuente (en `MPI_Recv`).
- **tag**: Un identificador numérico que ayuda a identificar el mensaje.
- **comm**: El comunicador, en este caso `MPI_COMM_WORLD`.
- **status**: Utilizado en `MPI_Recv` para almacenar información sobre el mensaje recibido.


### **4. Comunicación Colectiva (Collective Communication)**

Además de la comunicación punto a punto, MPI ofrece funciones de comunicación colectiva, donde múltiples procesos participan en la operación. Algunas funciones importantes incluyen:

- **MPI_Bcast**: Transmite un mensaje desde un proceso a todos los demás.
- **MPI_Scatter**: Divide los datos entre varios procesos.
- **MPI_Gather**: Recopila datos desde varios procesos.
- **MPI_Reduce**: Aplica una operación de reducción (como suma o multiplicación) sobre los datos recibidos desde todos los procesos y entrega el resultado a un proceso.

**Ejemplo: MPI_Bcast**

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    int number;
    if (world_rank == 0) {
        number = 100;  // El proceso 0 tiene el número original
    }
    MPI_Bcast(&number, 1, MPI_INT, 0, MPI_COMM_WORLD);  // Difunde el número a todos los procesos

    printf("Proceso %d recibió el número %d\n", world_rank, number);

    MPI_Finalize();
    return 0;
}
```

Este código utiliza `MPI_Bcast` para transmitir el valor del proceso 0 a todos los demás.

### **5. Trampas en la Paralelización con MPI**

Es importante evitar algunos errores comunes al paralelizar programas con MPI:

- **Condiciones de carrera**: Ocurren cuando varios procesos intentan acceder a los mismos recursos al mismo tiempo sin una adecuada sincronización.
- **Interbloqueos (Deadlocks)**: Ocurren cuando dos o más procesos esperan indefinidamente por mensajes del otro.
- **Sobrecarga de comunicación**: El uso excesivo de mensajes puede generar más tiempo de comunicación que de cálculo, reduciendo la eficiencia.

#### Ejemplo de Interbloqueo:

El siguiente código causa un interbloqueo porque ambos procesos están esperando recibir un mensaje antes de enviar uno:

```c
if (world_rank == 0) {
    MPI_Recv(&number, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    MPI_Send(&number, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
} else if (world_rank == 1) {
    MPI_Recv(&number, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    MPI_Send(&number, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
}
```

### **6. Topologías Virtuales**

#### **6.1 ¿Qué es una topología virtual?**

En sistemas de memoria distribuida, los procesadores pueden organizarse en diferentes formas (topologías) para facilitar la comunicación. Las topologías virtuales son estructuras que organizan a los procesos de manera lógica, simulando estructuras físicas como mallas, anillos o árboles. MPI provee funciones para definir y trabajar con estas topologías, lo que permite optimizar el paso de mensajes según la configuración lógica de los procesos.

#### **6.2 Tipos de Topologías**

- **Cartesiana (Malla o Grilla)**: Organiza los procesos en una malla 2D o 3D, lo que facilita la comunicación entre vecinos.
- **Anillo (Ring)**: Conecta cada proceso con el siguiente, formando un ciclo.

**Ejemplo: Topología en Malla 2D**

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank, world_size;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);

    // Definir una malla de 2x2 (si hay 4 procesos)
    int dims[2] = {2, 2};  // Dos dimensiones, cada una con 2 procesos
    int periods[2] = {0, 0};  // No hay periodicidad
    MPI_Comm cart_comm;
    MPI_Cart_create(MPI_COMM_WORLD, 2, dims, periods, 0, &cart_comm);

    int cart_rank;
    int coords[2];
    MPI_Comm_rank(cart_comm, &cart_rank);
    MPI_Cart_coords(cart_comm, cart_rank, 2, coords);

    printf("Proceso %d tiene coordenadas (%d, %d)\n", cart_rank, coords[0], coords[1]);

    MPI_Finalize();
    return 0;
}
```

En este ejemplo, los procesos se organizan en una malla de 2x2. Cada proceso puede averiguar sus coordenadas dentro de la malla mediante `MPI_Cart_coords`. Esta configuración permite definir vecinos de manera más eficiente cuando se trabaja con problemas en 2D o 3D, como simulaciones numéricas.


### **7. Tipos de Datos Derivados**

MPI permite definir tipos de datos derivados que consisten en estructuras complejas, como vectores, matrices o registros con diferentes tipos de datos. Esto es útil cuando necesitamos enviar estructuras más complejas que no pueden ser representadas por un único tipo de dato básico como `MPI_INT` o `MPI_FLOAT`.

**Ejemplo:** Definir y Enviar una Estructura

Supongamos que queremos enviar una estructura que contiene un entero y un flotante.

```c
#include <mpi.h>
#include <stdio.h>

typedef struct {
    int id;
    float value;
} data_t;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    // Definir el tipo derivado
    data_t data;
    if (world_rank == 0) {
        data.id = 42;
        data.value = 3.14;
    }

    // Definir el tipo de dato MPI para la estructura
    MPI_Datatype mpi_data_type;
    int lengths[2] = {1, 1};
    MPI_Aint displacements[2];
    MPI_Datatype types[2] = {MPI_INT, MPI_FLOAT};

    displacements[0] = offsetof(data_t, id);
    displacements[1] = offsetof(data_t, value);

    MPI_Type_create_struct(2, lengths, displacements, types, &mpi_data_type);
    MPI_Type_commit(&mpi_data_type);

    // Enviar y recibir la estructura
    if (world_rank == 0) {
        MPI_Send(&data, 1, mpi_data_type, 1, 0, MPI_COMM_WORLD);
    } else if (world_rank == 1) {
        MPI_Recv(&data, 1, mpi_data_type, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Proceso 1 recibió id = %d y value = %f\n", data.id, data.value);
    }

    MPI_Type_free(&mpi_data_type);
    MPI_Finalize();
    return 0;
}
```

Este ejemplo muestra cómo definir y enviar una estructura compleja usando `MPI_Type_create_struct`. Esto es útil cuando los datos que se deben comunicar son más complejos que los tipos primitivos.


### **8. MPI One-Sided Communication**

La comunicación de una sola cara, o "one-sided", permite que un proceso acceda directamente a la memoria de otro proceso sin que este último participe activamente en la comunicación en ese momento. Esto simplifica ciertos patrones de comunicación y puede mejorar el rendimiento en sistemas donde el acceso a memoria remota es eficiente.

Hay tres operaciones clave:
- **MPI_Put**: Copia datos desde el proceso emisor a la memoria del receptor.
- **MPI_Get**: Copia datos desde la memoria del receptor al emisor.
- **MPI_Accumulate**: Realiza operaciones de reducción (sumar, multiplicar, etc.) en la memoria remota.

**Ejemplo:** MPI_Put

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank, world_size;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);

    int data = 100;
    MPI_Win win;

    // Crear ventana de memoria
    MPI_Win_create(&data, sizeof(int), sizeof(int), MPI_INFO_NULL, MPI_COMM_WORLD, &win);

    if (world_rank == 0) {
        int value = 42;
        MPI_Win_lock(MPI_LOCK_EXCLUSIVE, 1, 0, win);  // Bloquear la ventana en el proceso 1
        MPI_Put(&value, 1, MPI_INT, 1, 0, 1, MPI_INT, win);  // Enviar valor al proceso 1
        MPI_Win_unlock(1, win);  // Desbloquear la ventana
        printf("Proceso 0 envió %d al proceso 1\n", value);
    }

    MPI_Win_fence(0, win);  // Sincronización

    if (world_rank == 1) {
        printf("Proceso 1 tiene data = %d\n", data);  // Debería imprimir 42
    }

    MPI_Win_free(&win);
    MPI_Finalize();
    return 0;
}
```

Este ejemplo usa `MPI_Put` para escribir directamente en la memoria del proceso 1 desde el proceso 0. La ventana de memoria compartida se crea con `MPI_Win_create`, y se usa `MPI_Win_lock` para asegurar que solo un proceso pueda escribir en la memoria a la vez.


### **9. MPI I/O**

MPI incluye soporte para entrada/salida paralela, lo que permite a los procesos leer y escribir archivos de manera eficiente en sistemas de archivos distribuidos. MPI I/O es especialmente útil cuando se manejan grandes volúmenes de datos en aplicaciones científicas, donde se requiere que varios procesos escriban o lean desde el mismo archivo.

#### Ejemplo: Escritura Paralela en un Archivo

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    MPI_File fh;
    MPI_Status status;

    // Abrir archivo en modo escritura
    MPI_File_open(MPI_COMM_WORLD, "output.txt", MPI_MODE_CREATE | MPI_MODE_WRONLY, MPI_INFO_NULL, &fh);

    char data[12];
    sprintf(data, "Proceso %d\n", world_rank);

    // Escribir datos al archivo
    MPI_File_write_at(fh, world_rank * 12, data, 12, MPI_CHAR, &status);

    MPI_File_close(&fh);
    MPI_Finalize();
    return 0;
}
```

En este ejemplo, cada proceso escribe su propio mensaje en el archivo "output.txt" en una posición específica. `MPI_File_write_at` asegura que cada proceso escribe en una posición diferente sin sobrescribir los datos de los otros procesos.


## **Ejercicios**

### **Ejercicio 1: Suma Paralela de Vectores**

**Descripción:** Implementa un programa MPI donde cada proceso genera un vector de números enteros de tamaño `N`. Luego, los procesos deben sumar sus vectores de forma paralela y almacenar el resultado en un proceso raíz (el proceso 0).

**Objetivo:** Aplicar la comunicación colectiva con `MPI_Reduce` y manejar datos distribuidos.

**Pasos:**
1. Cada proceso genera un vector de `N` elementos aleatorios.
2. Usa `MPI_Reduce` para sumar los vectores y almacenar el resultado en el proceso 0.
3. Imprime el resultado final desde el proceso 0.

**Pistas:**
- Utiliza `MPI_Reduce` con la operación `MPI_SUM`.
- Usa un tamaño de vector pequeño al inicio, como `N = 5`.


### **Ejercicio 2: Producto Matriz-Vector Paralelo**

**Descripción:** Implementa un programa MPI que realiza el producto de una matriz dispersa (distribuida entre varios procesos) y un vector, utilizando comunicación punto a punto.

**Objetivo:** Aplicar `MPI_Send` y `MPI_Recv` para distribuir la matriz entre los procesos, y calcular el resultado parcial en cada proceso.

**Pasos:**
1. Divide una matriz de tamaño `MxN` entre los procesos.
2. Cada proceso multiplica su submatriz por un vector global, y luego envía el resultado parcial al proceso raíz.
3. El proceso raíz debe reunir los resultados y mostrar el vector final.

**Pistas:**
- Usa una matriz pequeña (como 4x4) y un vector de tamaño 4.
- Distribuye las filas de la matriz entre los procesos.


### **Ejercicio 3: Implementación de una Topología de Malla 2D**

**Descripción:** Implementa una malla 2D utilizando MPI y distribuye datos entre los vecinos de la malla.

**Objetivo:** Aprender a utilizar `MPI_Cart_create` para crear una topología virtual y enviar datos entre procesos vecinos.

**Pasos:**
1. Crea una topología cartesiana de 2D con 4 procesos.
2. Asigna a cada proceso un identificador y sus coordenadas en la malla.
3. Envía datos desde un proceso a sus vecinos (norte, sur, este, oeste).

**Pistas:**
- Usa `MPI_Cart_create` para definir la topología.
- Aplica `MPI_Send` y `MPI_Recv` para la comunicación entre vecinos.


### **Ejercicio 4: Promedio Móvil de un Vector Usando Comunicación One-Sided**

**Descripción:** Implementa el cálculo de un promedio móvil en paralelo utilizando comunicación "one-sided" con MPI. El promedio móvil se calculará para un vector distribuido entre los procesos.

**Objetivo:** Aplicar `MPI_Put` para escribir directamente en la memoria de otros procesos y calcular el promedio móvil.

**Pasos:**
1. Cada proceso tiene una parte de un vector de números.
2. Cada proceso calcula el promedio de su valor actual y el de sus vecinos (izquierda y derecha).
3. Usa `MPI_Put` para escribir en la memoria de los procesos vecinos.

**Pistas:**
- Maneja cuidadosamente los bordes del vector, es decir, los procesos que están al inicio o al final del vector.


### **Ejercicio 5: Escritura Paralela en un Archivo con MPI I/O**

**Descripción:** Implementa un programa que distribuya una gran cantidad de datos entre varios procesos, y luego cada proceso escribe su porción en un archivo de manera paralela usando MPI I/O.

**Objetivo:** Aplicar `MPI_File_write_at` para permitir que varios procesos escriban simultáneamente en un archivo sin sobrescribir los datos.

**Pasos:**
1. Genera un array grande de enteros distribuidos entre varios procesos.
2. Cada proceso debe escribir su porción del array en un archivo común.
3. Usa `MPI_File_write_at` para asegurarte de que cada proceso escriba en su propia posición dentro del archivo.

**Pistas:**
- Usa un array de tamaño 100 elementos, donde cada proceso escribe en posiciones consecutivas.
- Verifica el archivo al final para confirmar que todos los procesos escribieron correctamente.
